## PreProcess

In [1]:
"""_summary_
"""
import requests
response = requests.get("http://mirror.coggle.club/seg_risky_training_anno.csv")
# 检查请求是否成功
if response.status_code == 200:
    with open("/home/tipriest/data/seg_risky/seg_risky_training_anno.csv", "wb") as file:
        file.write(response.content)
        print("文件下载成功")
else:
    print("文件下载失败")


文件下载成功


In [42]:
# !apt update > /dev/null; apt install aria2 git-lfs axel -y > /dev/null
# !pip install ultralytics==8.2.0 numpy pandas opencv-python Pillow matplotlib > /dev/null
# !axel -n 12 -a  http://mirror.coggle.club/seg_risky_training_data_01.zip -o ~/data/seg_risky/seg_risky_training_data_01.zip
# !unzip -q /home/tipriest/data/seg_risky/training_data_10.zip -d /home/tipriest/data/seg_risky/


### Import Libs

In [4]:
import os, shutil
import glob
import json
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
"""
这是一个示例模块。

此模块包含与示例功能相关的代码。
"""
BASE_DATA_PATH = "/home/tipriest/data/seg_risky"
# 一共有1020575的数据
training_anno = pd.read_csv(os.path.join(
    BASE_DATA_PATH, "seg_risky_training_anno.csv"))
train_jpgs = []
for i in range(0, 2, 1):
    train_jpgs += [x.replace('/home/tipriest/data/seg_risky/', '')
                    for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{i}/*.jpg'))]
training_anno = training_anno[training_anno['Path'].isin(train_jpgs)]
training_anno['Polygons'] = training_anno['Polygons'].apply(json.loads)


In [3]:
# df = pd.read_csv(os.path.join(
#     BASE_DATA_PATH, "seg_risky_training_anno.csv"))
# df[['Prefix', 'Suffix']] = df['Path'].str.split('/', expand=True)
# prefix_counts= df['Prefix'].value_counts()
# print(prefix_counts)
# for i in range(16):
#     folder_name = format(i, 'x')
#     folder_path = f"folder_{folder_name}"
#     train_jpgs = [x for x in glob.glob(os.path.join(BASE_DATA_PATH, f'{folder_name}/*.jpg'))]
#     print(i, len(train_jpgs))


Prefix
d    64212
3    64160
c    63942
a    63892
f    63881
2    63877
b    63870
6    63808
8    63789
0    63785
7    63725
4    63685
9    63628
5    63507
1    63453
e    63361
Name: count, dtype: int64


## Format dateset

In [4]:
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "datasetYOLO")
YOLO_SEG_DATASET_TRAIN_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "train/")
YOLO_SEG_DATASET_VALID_PATH = os.path.join(YOLO_SEG_DATASET_BASE_PATH, "valid/")

if os.path.exists(YOLO_SEG_DATASET_BASE_PATH):
    shutil.rmtree(YOLO_SEG_DATASET_BASE_PATH)
os.makedirs(YOLO_SEG_DATASET_TRAIN_PATH)
os.makedirs(YOLO_SEG_DATASET_VALID_PATH)

def normalize_polygon(polygon, img_width, img_height):
    normalized_polygon = [(min(x, img_width) / img_width, min(y, img_height) / img_height) for x, y in polygon]
    return normalized_polygon

for row in training_anno.iloc[500:5500].iterrows():
    shutil.copy(os.path.join(BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_TRAIN_PATH)
    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(YOLO_SEG_DATASET_TRAIN_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(polygon, img_width, img_height)
            normalized_coords = ' '.join([f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')

In [5]:
# 制作验证集III
for row in training_anno.iloc[:500].iterrows():
    shutil.copy(os.path.join(
        BASE_DATA_PATH, row[1].Path), YOLO_SEG_DATASET_VALID_PATH)

    img = cv2.imread(os.path.join(BASE_DATA_PATH, row[1].Path))
    img_height, img_width = img.shape[:2]
    txt_filename = os.path.join(
        YOLO_SEG_DATASET_VALID_PATH + row[1].Path.split('/')[-1][:-4] + '.txt')
    with open(txt_filename, 'w') as up:
        for polygon in row[1].Polygons:
            normalized_polygon = normalize_polygon(
                polygon, img_width, img_height)
            normalized_coords = ' '.join(
                [f'{coord[0]:.3f} {coord[1]:.3f}' for coord in normalized_polygon])
            up.write(f'0 {normalized_coords}\n')


In [3]:
YOLO_SEG_DATASET_BASE_PATH = os.path.join(BASE_DATA_PATH, "dataset")

In [8]:
with open(os.path.join(os.getcwd(), "data.yaml"), 'w') as up:
    data_root = os.path.abspath(YOLO_SEG_DATASET_BASE_PATH)
    up.write(f'''
path: {data_root}
train: train
val: valid

names:
    0: alter
''')

## Load Pretrained Model & Train model

In [10]:
# !wget http://mirror.coggle.club/yolo/Arial.ttf -O /home/tipriest/.cache/torch/hub/checkpoints/Arial.ttf
# !wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8n-seg-v8.2.0.pt -O /home/tipriest/.cache/torch/hub/checkpoints/yolov8n-seg.pt

In [ ]:
from ultralytics import YOLO
import os
model_base_path = "/home/tipriest/.cache/torch/hub/checkpoints"
# yolov8n-seg、yolov8s-seg、yolov8m-seg、yolov8l-seg和yolov8x-seg，模型的大小依次递增
# model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
model = YOLO(os.path.join(model_base_path, "yolo11x.pt"))
results = model.train(data=os.path.join(os.getcwd(), "data.yaml"), epochs=30, imgsz=256)

Ultralytics 8.3.11 🚀 Python-3.10.14 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)
engine/trainer: task=detect, mode=train, model=/home/tipriest/.cache/torch/hub/checkpoints/yolo11x.pt, data=/home/tipriest/Documents/AlgorithmPractice/src/deepLearning/DataWhale/Yolo/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False,

train: Scanning /home/tipriest/data/seg_risky/dataset/train.cache... 53877 images, 0 backgrounds, 18 corrupt: 100%|██████████| 53877/53877 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/20bc184a4ce6fb2b7da8292ea83b2673.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.046]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/20bf87ceeb9141bb1d23725b683e4b98.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.06]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/22bb834ee6967259179b65d875ac4756.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0059999]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/242e19afc691c5d9f60532593f605b1b.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.051]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/2484d02fcdda24fc246af4173cbc9318.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0220001]
train: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/train/24ad85f03d6f0a3282816


val: Scanning /home/tipriest/data/seg_risky/dataset/valid.cache... 10000 images, 0 backgrounds, 1 corrupt: 100%|██████████| 10000/10000 [00:00<?, ?it/s]

val: WARNING ⚠️ /home/tipriest/data/seg_risky/dataset/valid/2bec6ff53af2afd38998a727a2afd3cb.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.1060001]


Plotting labels to runs/detect/train2/labels.jpg... 


### Continue to train

In [7]:
from ultralytics import YOLO
model = YOLO("./runs/segment/train10/weights/best.pt")
results = model.train(data=os.path.join(YOLO_SEG_DATASET_BASE_PATH, "data.yaml"), epochs=80, imgsz=512)

/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_l

New https://pypi.org/project/ultralytics/8.3.13 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.0 🚀 Python-3.10.14 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 15959MiB)
engine/trainer: task=segment, mode=train, model=./runs/segment/train10/weights/best.pt, data=/home/tipriest/data/seg_risky/dataset/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=

/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/nn/tasks.py:732: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(file, map_l

AMP: checks passed ✅


/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/utils/checks.py:640: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/home/tipriest/anaconda3/envs/AIdefense/lib/python3.10/site-packages/ultralytics/engine/trainer.py:261: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.amp)
train: Scanning /home/tipriest/data/seg_risky/dataset/train... 53685 images, 0 backgrounds, 4 corrupt: 100%|██████████| 53685/53685 [00:21<00:00, 2474.59it/s]

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      22/80      5.19G     0.9308      1.004     0.7576      1.012          4        512: 100%|██████████| 3356/3356 [06:46<00:00,  8.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [00:36<00:00,  8.48it/s]

                   all       9998      14392      0.958      0.889      0.944      0.799      0.957      0.884      0.939      0.655



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      23/80      5.15G     0.9247     0.9997     0.7469      1.009         39        512:  81%|████████  | 2716/3356 [05:29<01:19,  8.10it/s]

## 训练后需要在这里加载最好的模型

In [10]:
from ultralytics import YOLO
import glob
from tqdm import tqdm
import os

model = YOLO("./runs/segment/train10/weights/best.pt")

test_imgs = glob.glob(os.path.join(BASE_DATA_PATH, "test_set_A_rename/*/*"))
# test_imgs = glob.glob(os.path.join(base_data_path, "data_set/train/0/*"))
# print(os.path.join(base_data_path, "data_set/train/0/*"))

In [11]:
import numpy as np
Polygon = []
for path in tqdm(test_imgs[:]):
    # print(path)
    results = model(path, verbose=False)
    result = results[0]
    if result.masks is None:
        Polygon.append([])
    else:
        image_polygons = []
        for mask in result.masks.xy:
            # 将掩膜的坐标转换为Numpy数组
            mask_coords = np.array(mask)
            # 获取外接矩形(bounding box)
            # print(mask_coords)
            # print()
            if 0 == mask_coords.size:
                continue
            x_min = round(float(np.min(mask_coords[:, 0])), 1)
            x_max = round(float(np.max(mask_coords[:, 0])), 1)
            y_min = round(float(np.min(mask_coords[:, 1])), 1)
            y_max = round(float(np.max(mask_coords[:, 1])), 1)
            bounding_box = [
                [x_min, y_min],
                [x_min, y_max],
                [x_max, y_max],
                [x_max, y_min]
            ]
            image_polygons.append(bounding_box)
            # print(bounding_box)
        Polygon.append(image_polygons)

100%|██████████| 100000/100000 [11:06<00:00, 149.95it/s]


In [12]:
import pandas as pd
submit = pd.DataFrame({
    'Path': [x.split('/')[-1] for x in test_imgs[:]],
    'Polygon': Polygon
})

In [13]:
submit = pd.merge(submit, pd.DataFrame({'Path': [x.split('/')[-1] for x in test_imgs[:]]}), on='Path', how='right')
submit = submit.fillna('[]')
submit.to_csv('track2_submit.csv', index=None)